In [3]:
import mlflow
import optuna.visualization as vis
import plotly.io as pio
import optuna
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, f1_score
from lightgbm import LGBMClassifier
from imblearn.over_sampling import ADASYN
from imblearn.pipeline import make_pipeline

# --- 1. Setup MLflow ---
# mlflow.set_tracking_uri("http://YOUR_URI_HERE") i m using dagshub because it is simple for me to ues because mlflow with ec2 mera kaam nhi  haha ha hehe
mlflow.set_experiment("EXP 6 - LightGBM Detailed HPT (ADASYN)")
optuna.logging.set_verbosity(optuna.logging.WARNING)

# --- 2. Load & Preprocess Data ---
try:
    df = pd.read_csv('reddit_preprocessed.csv')
except FileNotFoundError:
    print("❌ Error: 'reddit_preprocessed.csv' not found.")
    raise

# Drop missing values
df = df.drop('Unnamed: 0', axis=1, errors='ignore')
df = df.dropna()
df['clean_comment'] = df['clean_comment'].astype(str)

# ✅ FIX: Apply LabelEncoder
# This converts [-1, 0, 1] or ['neg', 'neu', 'pos'] -> [0, 1, 2]
le = LabelEncoder()
df['category_encoded'] = le.fit_transform(df['category'])

# Store mapping for later logging (e.g., 0=Negative, 1=Neutral, 2=Positive)
class_names = [str(cls) for cls in le.classes_]
print(f"✅ Labels Encoded: {list(zip(le.classes_, range(len(le.classes_))))}")

X = df['clean_comment']
y = df['category_encoded'] # Use the encoded labels

# Split Data (Stratify based on the NEW encoded labels)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- 3. Optuna Objective Function ---
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-4, 10.0, log=True),
        'n_jobs': 1,
        'random_state': 42,
        'verbose': -1,
        # Force multiclass objective
        'objective': 'multiclass',
        'num_class': len(class_names)
    }

    pipeline = make_pipeline(
        TfidfVectorizer(ngram_range=(1, 2), max_features=10000),
        ADASYN(random_state=42),
        LGBMClassifier(**params)
    )

    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='accuracy')

    return scores.mean()

# --- 4. Run Optimization ---
print("🚀 Starting Hyperparameter Tuning...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print(f"✅ Best Trial Accuracy: {study.best_value:.4f}")

# --- 5. Train Final Model ---
best_params = study.best_params
best_params['n_jobs'] = -1
best_params['random_state'] = 42
best_params['verbose'] = -1
best_params['objective'] = 'multiclass'
best_params['num_class'] = len(class_names)

final_pipeline = make_pipeline(
    TfidfVectorizer(ngram_range=(1, 2), max_features=10000),
    ADASYN(random_state=42),
    LGBMClassifier(**best_params)
)

print("\n🔄 Training Final Model...")
final_pipeline.fit(X_train, y_train)
y_pred = final_pipeline.predict(X_test)

# --- 6. Log Results ---
with mlflow.start_run(run_name="Best_LightGBM_ADASYN_Encoded_2"):
    mlflow.log_params(best_params)
    mlflow.log_param("features", 10000)

    # Use target_names to map 0,1,2 back to real names in the report
    report_dict = classification_report(y_test, y_pred, target_names=class_names, output_dict=True)
    report_str = classification_report(y_test, y_pred, target_names=class_names)

    print("\n📊 Final Classification Report:")
    print(report_str)

    mlflow.log_metric("accuracy", report_dict['accuracy'])
    mlflow.log_metric("macro_f1", report_dict['macro avg']['f1-score'])

    # Log by class name (e.g., class_Positive_f1, class_Negative_f1)
    for label, metrics in report_dict.items():
        if isinstance(metrics, dict) and label not in ['macro avg', 'weighted avg', 'accuracy']:
             # Use the actual class name in the log key
            mlflow.log_metric(f"class_{label}_precision", metrics['precision'])
            mlflow.log_metric(f"class_{label}_recall", metrics['recall'])
            mlflow.log_metric(f"class_{label}_f1", metrics['f1-score'])

    #Log Visualization (Optional)
    try:
        fig = optuna.visualization.plot_param_importances(study)
        fig.write_image("param_importance.png")
        mlflow.log_artifact("param_importance.png")
    except:
        pass
    # 1. Generate the plots
    fig_history = vis.plot_optimization_history(study)
    fig_importance = vis.plot_param_importances(study)
    fig_slice = vis.plot_slice(study)

    # 2. Show them immediately in Colab
    print("📊 Optimization History:")
    fig_history.show()

    print("\n📊 Parameter Importance:")
    fig_importance.show()

print("✅ Experiment Complete.")

✅ Labels Encoded: [(np.int64(-1), 0), (np.int64(0), 1), (np.int64(1), 2)]
🚀 Starting Hyperparameter Tuning...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/ut

✅ Best Trial Accuracy: 0.8546

🔄 Training Final Model...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



📊 Final Classification Report:
              precision    recall  f1-score   support

          -1       0.80      0.76      0.78      1650
           0       0.83      0.97      0.90      2529
           1       0.91      0.82      0.86      3154

    accuracy                           0.86      7333
   macro avg       0.85      0.85      0.85      7333
weighted avg       0.86      0.86      0.86      7333



/usr/local/lib/python3.12/dist-packages/kaleido/_sync_server.py:11: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.




📊 Optimization History:



📊 Parameter Importance:


🏃 View run Best_LightGBM_ADASYN_Encoded_2 at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/6/runs/02722abc34534b158ddbcafd0925a2cc
🧪 View experiment at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/6
✅ Experiment Complete.


In [1]:
!pip install mlflow dagshub optuna lightgbm imbalanced-learn kaleido

import optuna.visualization as vis
import plotly.io as pio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.2/263.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 k

In [2]:
import dagshub
dagshub.init(repo_owner='Ritk-Raikwar', repo_name='reddit-comment-sentiment-analysis', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=0e542771-ed85-4ad9-b3e5-54af7611cf5c&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=cec9207c1c1016390c882a950c89931a79dd99afad952694265233fc35071675




Accessing as Ritk-Raikwar

Initialized MLflow to track repo "Ritk-Raikwar/reddit-comment-sentiment-analysis"

Repository Ritk-Raikwar/reddit-comment-sentiment-analysis initialized!